In [1]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
import keras 
import nltk
%matplotlib inline

Using TensorFlow backend.


Traigo el DataFrame

In [2]:
from google.colab import files

In [3]:
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving twiter_with_data_location.csv to twiter_with_data_location.csv


In [4]:
tw = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')), index_col=0)
tw_test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')), index_col=0)
twiter_my=pd.read_csv(io.StringIO(uploaded['twiter_with_data_location.csv'].decode('utf-8')), index_col=0)

In [5]:
#quito los espacios en blando que poseen los nombres de los paises paises
twiter_my["country"]=twiter_my.loc[twiter_my.location_free.isnull()==False]["country"].transform(lambda x: x.str.lstrip())

In [42]:
tw["long_text"]=tw.text.apply(lambda x: len(x))
tw["long_text"]=tw.text.apply(lambda x: len(x))

Veo los StopWords

In [113]:
import re, string

def remove_punctuation( text ):
  return re.sub('[%s]' % re.escape(string.punctuation), ' ', text)

In [114]:
tw.text=tw.text.apply(lambda x: remove_punctuation(x))

TypeError: ignored

In [7]:
#esta libreria me permite ver los stop words para el idioma English
from nltk.corpus import stopwords

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
#Traigo los StopWords en el englis
en_stops = set(stopwords.words('english'))

In [10]:
def remove_stopWord(texts):
  for word in texts:
    if word in en_stops:
      texts.remove(word)
  return texts

In [11]:
#paso a minuscula
tw["text"]=tw.text.str.lower()

In [12]:
#separo cada en un vector de array las palabras de cada texto
tw["text"]=tw.text.str.split()

In [13]:
#remuevo los stopWords con mi funcion remove_stopWord
tw["text"]=tw["text"].apply(remove_stopWord)
tw["text"]=tw["text"].apply(remove_stopWord)

In [14]:
tw["text"].head(2)

id
1    [deeds, reason, #earthquake, may, allah, forgi...
4       [forest, fire, near, la, ronge, sask., canada]
Name: text, dtype: object

Aplico Word2Vect

In [15]:
!pip install gensim

In [16]:
import gensim

In [17]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [18]:
#wprd=[['123asd',"teerees"]]
#wtv_model1=Word2Vec(wprd,min_count=1)
#wtv_model1.wv.vocab
#wprd=[['ssss',"aaaa"]]
#wtv_model1.build_vocab(wprd, update=True)
#total_examples
#wtv_model1.train(wprd,total_examples=1, epochs=1)
#wtv_model1.wv.vocab

In [19]:
models_wv=Word2Vec(tw.text,min_count=1)

In [ ]:
models_wv.wv.vocab

In [21]:
#agrego al vocab de models_wv los paises
#models_wv.build_vocab(tw.long_text,update=True)
#models_wv.train(tw.long_text,total_examples=1,epoch=1)

In [43]:
tw.head(2)

,keyword,location,text,target,long_text
id,,,,,
1,NaN,NaN,"[deeds, reason, #earthquake, may, allah, forgi...",1,7
4,NaN,NaN,"[forest, fire, near, la, ronge, sask., canada]",1,7


Separo mis datos de entrenamiento

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X=tw.text
y=tw.target

In [46]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

**Usanado Keras**

In [47]:
#con mi modelo de word2vect voy a transformat mis datos

In [48]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [49]:
tokenizer = Tokenizer(num_words=len(tw.text))

In [50]:
tokenizer.fit_on_texts(tw.text)

In [51]:
sequences = tokenizer.texts_to_sequences(tw.text)

In [52]:
vocab_size = len(tokenizer.word_index) + 1

In [53]:
#obtengo la longitud maxima de palabras en los textos
long_max=tw.long_text.max()
long_max

28

In [54]:
def getVector(word):
  vew=None
  if word in models_wv:
    vew=models_wv.wv[word]
  return vew
def isInModel(word):
 return word in models_wv

In [55]:
embeddings_index = {}
for w in models_wv.wv.vocab.keys():
    embeddings_index[w] = models_wv.wv[w]

In [56]:
embedding_matrix={}
for word,i in tokenizer.word_index.items():
  vector_ofWord = embeddings_index.get(word)
  if vector_ofWord is not None:
    embedding_matrix[i] = vector_ofWord

In [57]:
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)

In [58]:
X_train=pad_sequences(X_train, maxlen=long_max)
X_test=pad_sequences(X_test, maxlen=long_max)

In [ ]:
X_test[:5]

**Aplicando Keras**

In [59]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Embedding,Activation
from keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D
from keras import backend as K

In [60]:
len(embedding_matrix)

27923

In [61]:
model = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model.add(e)
model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
model.fit(X_train, y_train,
          validation_data=(X_test, y_test), 
          epochs=5, 
          batch_size=28,
          verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.5484 - accuracy: 0.7212 - val_loss: 0.4751 - val_accuracy: 0.7820
Epoch 2/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.3085 - accuracy: 0.8791 - val_loss: 0.5146 - val_accuracy: 0.7610
Epoch 3/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.1791 - accuracy: 0.9374 - val_loss: 0.6576 - val_accuracy: 0.7617
Epoch 4/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.1261 - accuracy: 0.9599 - val_loss: 0.6690 - val_accuracy: 0.7590
Epoch 5/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.0950 - accuracy: 0.9693 - val_loss: 0.7433 - val_accuracy: 0.7610


In [70]:
model2 = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model2.add(e)
model2.add(Conv1D(kernel_size=2,filters=100,padding='valid',activation="relu", strides=1))
model2.add(Conv1D(100, kernel_size=2, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dropout(0.25))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 200)           5584600   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 27, 100)           40100     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 5,650,813
Trainable params: 5,650,813
Non-trainable params: 0
____________________________________________

In [72]:
model2.fit(X_train, y_train,
          validation_data=(X_test, y_test), 
          epochs=7, 
          batch_size=28,
          verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.5676 - accuracy: 0.7057 - val_loss: 0.5073 - val_accuracy: 0.7669
Epoch 2/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.3361 - accuracy: 0.8665 - val_loss: 0.5697 - val_accuracy: 0.7577
Epoch 3/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.1923 - accuracy: 0.9328 - val_loss: 0.6635 - val_accuracy: 0.7643
Epoch 4/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.1258 - accuracy: 0.9593 - val_loss: 0.7491 - val_accuracy: 0.7636
Epoch 5/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.0957 - accuracy: 0.9691 - val_loss: 0.7680 - val_accuracy: 0.7472
Epoch 6/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.0727 - accuracy: 0.9755 - val_loss: 0.9140 - val_accuracy: 0.7446
Epoch 7/7
6090/6090 [==============================] - 17s 3ms/step - loss: 0.0644 - acc

In [67]:
model_cnn_01 = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(1, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_01.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=28, verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/5
 - 15s - loss: 0.5418 - accuracy: 0.7195 - val_loss: 0.4756 - val_accuracy: 0.7820
Epoch 2/5
 - 15s - loss: 0.2865 - accuracy: 0.8874 - val_loss: 0.5554 - val_accuracy: 0.7663
Epoch 3/5
 - 15s - loss: 0.1568 - accuracy: 0.9489 - val_loss: 0.6133 - val_accuracy: 0.7571
Epoch 4/5
 - 15s - loss: 0.1080 - accuracy: 0.9672 - val_loss: 0.6664 - val_accuracy: 0.7610
Epoch 5/5
 - 15s - loss: 0.0795 - accuracy: 0.9737 - val_loss: 0.7530 - val_accuracy: 0.7597


In [68]:
model3 = Sequential()
e = Embedding(len(embedding_matrix), 200, input_length=28)
model3.add(e)
model3.add(Conv1D(kernel_size=2,filters=100,padding='valid',activation="relu", strides=1))
model3.add(Conv1D(100, kernel_size=2, activation='relu'))
model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.25))
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(100, activation='relu'))
model3.add(Dropout(0.75))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
model3.fit(X_train, y_train,
          validation_data=(X_test, y_test), 
          epochs=5, 
          batch_size=28,
          verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 17s 3ms/step - loss: 0.6447 - accuracy: 0.6236 - val_loss: 0.5315 - val_accuracy: 0.7380
Epoch 2/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.4288 - accuracy: 0.8245 - val_loss: 0.4867 - val_accuracy: 0.7722
Epoch 3/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.2594 - accuracy: 0.9090 - val_loss: 0.6231 - val_accuracy: 0.7610
Epoch 4/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.1516 - accuracy: 0.9507 - val_loss: 0.7379 - val_accuracy: 0.7544
Epoch 5/5
6090/6090 [==============================] - 16s 3ms/step - loss: 0.1100 - accuracy: 0.9629 - val_loss: 0.9075 - val_accuracy: 0.7617


**Probandolo en el set**


In [115]:
def creando_DF(x_test_seq,test):
  yhat_cnn = model2.predict_classes(x_test_seq)
  submit = test['id'].to_frame()
  submit.to_csv('submit.csv', index=False)
  files.download('submit.csv')

In [ ]:
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)
X_train=pad_sequences(X_train, maxlen=long_max)
X_test=pad_sequences(X_test, maxlen=long_max)

In [85]:
uploaded2 = files.upload()

Saving test.csv to test (1).csv


In [86]:
test = pd.read_csv(io.StringIO(uploaded2['test.csv'].decode('utf-8')))

In [87]:

x_test=test["text"]
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=28)

In [88]:
test.head(2)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."


In [89]:
yhat_cnn = model2.predict_classes(x_test_seq)

In [91]:
submit = test['id'].to_frame()

In [92]:
submit

,id
0,0
1,2
2,3
3,9
4,11
...,...
3258,10861
3259,10865
3260,10868
3261,10874


In [93]:
submit['target'] = pd.DataFrame(yhat_cnn)

In [94]:
submit

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [97]:

submit.to_csv('submit.csv', index=False)
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [98]:
yhat_cnn3 = model3.predict_classes(x_test_seq)

In [107]:
submit3 = test['id'].to_frame()

In [108]:
submit3['target'] = pd.DataFrame(yhat_cnn3)

In [109]:

submit.to_csv('submit3.csv', index=False)
files.download('submit3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>